In [5]:
#LSA(한글뉴스)
import pandas as pd

f = open('c:/data/text/news1.txt', 'r', encoding='utf-8')

lines = f.readlines()
f.close()

In [6]:
import re
from konlpy.tag import Hannanum

han = Hannanum()
word_list = []
for i in range(len(lines)):
    sentence = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', lines[i])
    #명사만 추출
    a = sentence.strip()
    n = han.nouns(a)
    n2 = [x for x in n if len(x) > 1]
    if len(n2) > 0:
        word_list.append(n2)

In [7]:
#tf-idf 행렬을 만들기 위해 역토큰화
detokenized_doc = []
for i in range(len(word_list)):
    t = ' '.join(word_list[i])
    detokenized_doc.append(t)

detokenized_doc

['이데일리 이정현 김진국 청와대 민정수석 아들 입사지원서 논란 하루만 사의 표명 문재 대통령 수용',
 '아들 입사지원서 문제 논란 김진국 청와대 민정수석 사의 표명 문재 대통령 수석 사의 수용 청와대 고위관계자 이날 기자들 수석 오늘 아침 출근 사의 사진 수석 청와대 수석 보좌관회의 참석 모습 연합뉴스',
 '박수현 청와대 국민소통수석 이날 춘추관 브리핑 대통령 수석 사의 수용',
 '청와대 고위 관계자 기자들 수석 출근 사의 대통령 수용 수석 이날 예정 국무회의 참석',
 '대통령의 수용 배경 사정 국민 청와대 부응 취지 설명 사의수용 과정 대통령의 언급',
 '관계자 민정수석 임명 논의 계획',
 '수석 아들 김모 취업 기업 제출 입사지원서 아버지 민정수석 내용 논란']

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vertorizer = TfidfVectorizer()
X = vertorizer.fit_transform(detokenized_doc)
X.shape

(7, 54)

In [10]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=17)
svd_model.fit(X)
len(svd_model.components_)

7

In [11]:
import numpy as np
#토픽수 x 단어수
np.shape(svd_model.components_)

(7, 54)

In [12]:
svd_model.components_

array([[ 0.03559658,  0.10923969,  0.07534182,  0.01778798,  0.12022654,
         0.10923969,  0.01778798,  0.1136015 ,  0.06196306,  0.15321851,
         0.06196306,  0.14503735,  0.06196306,  0.16793776,  0.03559658,
         0.24490892,  0.03557597,  0.07534182,  0.14503735,  0.07534182,
         0.16773306,  0.1136015 ,  0.01778798,  0.07534182,  0.01778798,
         0.1136015 ,  0.33773297,  0.01778798,  0.01778798,  0.07534182,
         0.01778798,  0.4283868 ,  0.22413204,  0.16793776,  0.06196306,
         0.07534182,  0.01778798,  0.07534182,  0.10923969,  0.07534182,
         0.21156976,  0.09938389,  0.09938389,  0.03559658,  0.16793776,
         0.06196306,  0.15321851,  0.30544331,  0.1136015 ,  0.15321851,
         0.06196306,  0.01778798,  0.14503735,  0.09938389],
       [ 0.19596235, -0.08495265, -0.00610439, -0.08131961,  0.09214758,
        -0.08495265, -0.08131961, -0.15401796,  0.21251486, -0.07558517,
         0.21251486,  0.08397261,  0.21251486,  0.22256265,  0.

In [13]:
#단어 집합
terms = vertorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print('Topic %d:' % (idx + 1),
        [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:1]])

get_topics(svd_model.components_,terms)

Topic 1: [('사정', 0.01779), ('취지', 0.01779), ('설명', 0.01779), ('과정', 0.01779), ('부응', 0.01779), ('배경', 0.01779), ('국민', 0.01779), ('언급', 0.01779), ('사의수용', 0.01779), ('대통령의', 0.03558), ('계획', 0.0356), ('논의', 0.0356), ('임명', 0.0356), ('내용', 0.06196), ('취업', 0.06196), ('기업', 0.06196), ('김모', 0.06196), ('제출', 0.06196), ('아버지', 0.06196), ('사진', 0.07534), ('고위관계자', 0.07534), ('보좌관회의', 0.07534), ('문제', 0.07534), ('모습', 0.07534), ('연합뉴스', 0.07534), ('오늘', 0.07534), ('아침', 0.07534), ('이정현', 0.09938), ('이데일리', 0.09938), ('하루만', 0.09938), ('고위', 0.10924), ('예정', 0.10924), ('국무회의', 0.10924), ('브리핑', 0.1136), ('춘추관', 0.1136), ('박수현', 0.1136), ('국민소통수석', 0.1136), ('관계자', 0.12023), ('표명', 0.14504), ('김진국', 0.14504), ('문재', 0.14504), ('기자들', 0.15322), ('참석', 0.15322), ('출근', 0.15322), ('민정수석', 0.16773), ('입사지원서', 0.16794), ('아들', 0.16794), ('논란', 0.16794)]
Topic 2: [('이날', -0.17389), ('대통령의', -0.16264), ('브리핑', -0.15402), ('춘추관', -0.15402), ('박수현', -0.15402), ('국민소통수석', -0.15402), ('청와대', -0.12483), (